"rental_list_2022.csv" 파일을 불러와 다음 문제를 풀이하시오.

In [1]:
import pandas as pd
df = pd.read_csv("rental_list_2022.csv")
df.head(2)

id  host_id     room_type  price  minimum_nights  number_of_reviews  \
0  106332   264345  Private room     55               3                 65   
1   13913    54730  Private room     50               1                 30   

  last_review  reviews_per_month  availability_365  
0  2016-06-10               0.47                74  
1  2022-07-15               0.20               343

### Q1. id는 숙소의 식별자이고 host_id는 호스트의 식별자라고 한다. 두 변수를 기반으로 각 호스트가 몇 개의 숙소를 등록했는지 파악하고 2개 이상의 숙소를 관리하는 것으로 추정되는 호스트는 총 몇 명인가?
(정답 예시: 123)

In [2]:
df_ids = df[["id", "host_id"]].drop_duplicates()
df_ids.head(2)

id  host_id
0  106332   264345
1   13913    54730

In [5]:
df_host_cnt = df_ids["host_id"].value_counts().reset_index()
df_host_cnt.head(2)

index  host_id
0  28820321      285
1  33889201      266

In [6]:
sum(df_host_cnt["host_id"] >= 2)

7724

### Q2. room_type별 숙박비의 기술통계량을 정리하고자 한다. 다음과 같은 형식으로 정리하시오.
※ 평균, 중앙값, 최소값, 최대값, 표준편차를 정리하시오.  
※ 형식 예시는 다음과 같다.
![테이블 예시](test_stat_table_example_01.png)

In [10]:
df_stats = df.groupby("room_type")["price"].agg(["min", "median", "mean", "max", "std"])
df_stats = df_stats.reset_index()
df_stats.columns = ["room_type"] + ("price_" + df_stats.columns[1:]).to_list()
df_stats

room_type  price_min  price_median  price_mean  price_max   price_std
0  Entire home/apt          8         149.0  227.138778      20362  444.387421
1       Hotel room          0         202.0  245.836066       1490  233.000640
2     Private room          0          50.0  103.326067      25000  351.886950
3      Shared room          7          39.0   66.235149       1000   94.050626

### Q3. 월평균 후기 개수(reviews_per_month) 변수와 년중 이용 가능일수(availability)의 상관관계를 확인하고자 한다. 이를 위해 Pearson 상관분석을 실시하고 그 결과를 출력하시오.
※ scipy 라이브러리의 pearsonr() 함수를 사용하시오.  
※ 상관계수를 출력하시오. 단, 유의수준 5% 기준으로 귀무가설을 기각하지 못하는 경우 P-value를 출력하시오.  
※ 결과는 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [13]:
from scipy.stats import pearsonr

In [17]:
df_sub = df.dropna()
stat, p = pearsonr(df_sub["reviews_per_month"], df_sub["availability_365"])
if p > 0.05:
    print(round(p, 2))
else: 
    print(round(stat, 2))

0.24


### Q4. 호스트의 1년 수입을 대략적으로 추정해보고자 한다. 각 숙소별 1년 기대수익을 계산하고 이를 호스트별로 합산하시오. 호스트의 1년 기대수익의 평균은 얼마인가?
※ 단, "room_type"이 "Private room"인 숙소를 대상으로 분석한다.  
※ 기대수익 계산식은 다음과 같다.  
$$기대수익 = price \times availability\_365 \times 0.3$$

※ 정답은 반올림하여 소수점 둘째 자리까지 기술하시오.  
(정답 예시: 1.23)


In [34]:
df["rev"] = df["price"] * df["availability_365"] * 0.3
df_sub = df.loc[df["room_type"] == "Private room", ]
df_host_rev = df_sub.groupby("host_id")["rev"].sum().reset_index()
df_host_rev.head(2)

host_id      rev
0     4775  30281.4
1     6774     45.0

In [35]:
round(df_host_rev["rev"].mean(), 2)

6182.78

### Q5. room_type별 숙박비의 평균이 유의미하게 차이나는지 확인하고자 한다. 이를 위해 일원분산분석을 실시하고 그 결과를 확인하시오.
※ 마지막 후기 작성이 2020년 이전인 데이터는 분석에서 제외한다.  
※ 검정통계량을 출력하시오. 단, 유의수준 5% 기준으로 귀무가설을 기각하지 못하는 경우 P-value을 출력하시오.  
※ 결과는 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [24]:
df["last_review"] = pd.to_datetime(df["last_review"])
df["year"] = df["last_review"].dt.year

In [27]:
df_sub = df.loc[df["year"] >= 2020, ]
df_sub.head(2)

id  host_id        room_type  price  minimum_nights  number_of_reviews  \
1   13913    54730     Private room     50               1                 30   
2  284532  1481412  Entire home/apt     90               5                 42   

  last_review  reviews_per_month  availability_365     rev    year  
1  2022-07-15               0.20               343  5145.0  2022.0  
2  2022-01-04               0.34               222  5994.0  2022.0

In [28]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [32]:
model = ols(formula = "price ~ room_type", data = df_sub).fit()
model_result = anova_lm(model)
model_result

df        sum_sq       mean_sq           F  PR(>F)
room_type      3.0  1.589453e+08  5.298177e+07  566.707318     0.0
Residual   37353.0  3.492152e+09  9.349053e+04         NaN     NaN

In [33]:
if model_result["PR(>F)"][0] > 0.05:
    print(round(model_result["PR(>F)"][0], 2))
else:
    print(round(model_result["F"][0], 2))

566.71
